<a href="https://colab.research.google.com/github/ipavlopoulos/toxic_spans/blob/master/Unsupervised_Toxicity_Mitigation_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the data

In [3]:
import pandas as pd
cctk = pd.read_csv("cctk.train.csv.zip") 

Downloading...
From: https://drive.google.com/uc?id=1gSdQCIqVZTb8oGrsBL0m8zcJCW9I3c5x
To: /content/cctk.train.csv.zip
286MB [00:02, 139MB/s]


# Build a classifier (wrapper)
* Any method could be used inside this wrapper
* It should simply implement tokenise, train & predict

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.pipeline import make_pipeline

class clf:
  def __init__(self, tokenise = lambda x: x.split()):
    self.tokenise = tokenise
    self.vectorizer = CountVectorizer(analyzer='word',
                                      tokenizer=tokenise,
                                      binary=True)
    self.model = LogisticRegression(n_jobs=1, C=1e5)
  
  def train(self, X_train, y_train, X_dev=None, y_dev=None):
    train_vectors = self.vectorizer.fit_transform(X_train)
    self.model.fit(train_vectors, y_train)
    # uncomment below to use devevlopment data
    #val_preds = self.model.predict(vectorizer.transform(X_dev))

  def predict(self, x):
    return self.model.predict(self.vectorizer.transform(x))

- Use LR as a usecase

In [229]:
from sklearn.metrics import *
from sklearn.model_selection import train_test_split

test_size = 10000

sample = cctk.sample(test_size*10, random_state=42)
sample["label"] = sample.target.apply(round)
train, test = train_test_split(sample, test_size=test_size)

X_train, X_val, y_train, y_val = train_test_split(train.comment_text, train.label, test_size=test_size, random_state=40)
classifier = clf()
classifier.train(X_train, y_train)
predictions = classifier.predict(X_val)

accuracy = accuracy_score(y_val, predictions)
precision = precision_score(y_val, predictions, average='weighted')
recall = recall_score(y_val, predictions, average='weighted')
f1 = f1_score(y_val, predictions, average='weighted')
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))
print(f"ROC: {roc_auc_score(y_val, predictions)}")

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


accuracy = 0.945, precision = 0.941, recall = 0.945, f1 = 0.943
ROC: 0.7141060501608211


# Use Input Erasure


In [163]:
!git clone https://github.com/ipavlopoulos/toxic_spans.git

Cloning into 'toxic_spans'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 2), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [0]:
from toxic_spans.InputErasure import models

# choose a correctly predicted toxic example
show_cases = X_val.iloc[np.where((y_val==1)&(predictions==1))]


In [233]:
# select an example
case = show_cases.sample(1).to_list()[0]
# apply input erasure
input_mask = models.Mask(classifier, case)
mitigated_text = input_mask.get_mitigated_text()
# show
print(f"THE TEXT:\n{case}\n")
print(f"THE Mitigated TEXT:\n{mitigated_text}")

THE TEXT:
A fool and his money are soon parted, sciguybm. Have you been donating to greenpeace?

THE Mitigated TEXT:
A [mask] and his money are soon parted, sciguybm. Have you been donating to greenpeace?
